In [1]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
from langdetect import DetectorFactory, detect
from langdetect.lang_detect_exception import LangDetectException
DetectorFactory.seed = 0

def is_english(text):
    try:
        if detect(text) != "en":
            return False
    except LangDetectException:
        return False
    return True

def scrape_tweets(keyword_search:str, since_date:str, until_date:str):
    tweets = []
    for j, tweet in enumerate(sntwitter.TwitterSearchScraper(keyword_search+' since:'+since_date+' until:'+until_date).get_items()):
#        if j>100000:
#           break
        tweets.append([tweet.date, tweet.id, tweet.content, tweet.user.username])
    tweets_df = pd.DataFrame(tweets, columns=['Datetime', 'Tweet Id', 'Text', 'Username'])
    tweets_df['language'] =  tweets_df['Text'].apply(is_english)
    tweets_df = tweets_df[tweets_df['language'] == True].reset_index()
    return tweets_df

In [2]:
tweets_df = scrape_tweets("cmv transplant", "2022-01-01", "2022-07-07")
tweets_df.shape

(287, 6)

In [7]:
tweets_df.shape
#transplant infectio

(287, 12)

In [11]:
tweets_df.tail()

,index,Datetime,Tweet Id,Text,Username,language,transplant,maribavir,health_care,burden,care,financial
282,284,2022-01-05 00:13:03+00:00,1478519880538275845,CMV exposure drives long-term CD57+ CD4 memory...,ManchesterCytof,True,,,,,,
283,285,2022-01-04 22:36:59+00:00,1478495704901427201,#TxID #IDtwitter \n\nAre your transplant cente...,Ashka_TxID,True,,,,,,
284,286,2022-01-04 09:24:36+00:00,1478296294565105665,Yeh et al found that cytomegalovirus (#CMV) dr...,GaneshManjeri,True,,,,,,
285,287,2022-01-04 02:02:23+00:00,1478185006220595200,The most common opportunistic infection in kid...,Rx_Ed,True,,,,,,
286,288,2022-01-03 22:41:43+00:00,1478134504908808195,"""Use of Letermovir-Valganciclovir combination ...",CliniTransplant,True,,,,,,


In [13]:
len(tweets_df['index'])

287

In [6]:
from transformers import pipeline

tweets_df['transplant'] = ''
tweets_df['maribavir'] = ''
tweets_df['health_care'] = ''
tweets_df['burden'] = ''
tweets_df['care'] = ''
tweets_df['financial'] = ''

classifier = pipeline("zero-shot-classification")

for tweets in len(tweets_df['index']):
    print(tweets)
    a = classifier(
        tweets_df['Text'][tweets],
        candidate_labels=["transplant", "maribavir", "health care", "burden", "care", "financial"],)
    
    a = pd.DataFrame.from_dict(a)
    
    tweets_df['transplant'][1] = a.iloc[0,2]
    tweets_df['maribavir'][1] = a.iloc[1,2]
    tweets_df['health_care'][1] = a.iloc[2,2]
    tweets_df['burden'][1] = a.iloc[3,2]
    tweets_df['care'][1] = a.iloc[4,2]
    tweets_df['financial'][1] = a.iloc[5,2]

No model was supplied, defaulted to roberta-large-mnli (https://huggingface.co/roberta-large-mnli)
All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at roberta-large-mnli.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


0


C:\Users\Woon.Kim\AppData\Local\Temp\ipykernel_12984\3703963135.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets_df['transplant'][1] = a.iloc[0,2]
C:\Users\Woon.Kim\AppData\Local\Temp\ipykernel_12984\3703963135.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets_df['maribavir'][1] = a.iloc[1,2]
C:\Users\Woon.Kim\AppData\Local\Temp\ipykernel_12984\3703963135.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets_

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279

KeyError: 287

In [ ]:
a = classifier(

        tweets_df['Text'][0],

        candidate_labels=["transplant", "maribavir", "health care", "burden", "care", "financial"],

)

In [ ]:
a

In [ ]:
from transformers import pipeline

tweets_df['transplant'] = ''
tweets_df['maribavir'] = ''
tweets_df['health_care'] = ''
tweets_df['burden'] = ''
tweets_df['care'] = ''
tweets_df['financial'] = ''

classifier = pipeline("zero-shot-classification")
a = classifier(
    tweets_df['Text'][1],
    candidate_labels=["transplant", "maribavir", "health care", "burden", "care", "financial"],)

a = pd.DataFrame.from_dict(a)


tweets_df['transplant'][1] = a.iloc[0,2]
tweets_df['maribavir'][1] = a.iloc[1,2]
tweets_df['health_care'][1] = a.iloc[2,2]
tweets_df['burden'][1] = a.iloc[3,2]
tweets_df['care'][1] = a.iloc[4,2]
tweets_df['financial'][1] = a.iloc[5,2]

tweets_df